In [1]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-py3-none-any.whl size=16035 sha256=bd24d004dde6149ec69d2a51bd3b448575a99600e8b7262c9e1f10873c5447a1
  Stored in directory: /root/.cache/pip/wheels/b7/cc/0d/41d384b0071c6f46e542aded5f8571700ace4f1eb3f1591c29
Successfully built efficientnet-pytorch


In [2]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms
import torch.nn.functional as F
from efficientnet_pytorch import EfficientNet
from PIL import Image

from pathlib import Path 
from tqdm.notebook import tqdm

In [3]:
train_path = '../input/chest-xray-covid19-pneumonia/Data/train/'
test_path = '../input/chest-xray-covid19-pneumonia/Data/test/'

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
def path2files(path, pattern='*/*'):
    files = []
    for file_names in Path(path).glob(pattern):
        files.append(file_names)
        
    return files

In [5]:
class LoadData(Dataset):
    def __init__(self, posix_path_list, transform=None):
        super().__init__()
        self.file_names = posix_path_list
        self.transform = transform
        
    def __getitem__(self, idx): 
        file = self.file_names[idx]
        img = Image.open(file).convert('RGB')
        
        if self.file_names[idx].parent.stem == 'NORMAL':
            target = torch.tensor([0.0])
        else:
            target = torch.tensor([1.0])
        
        if self.transform:
            img = self.transform(img)           
            return img, target
        else: 
            return img, target
    
    def __len__(self): return len(self.file_names)

In [6]:
train = path2files(train_path)
val = path2files(test_path)

In [7]:
tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (1.0, 1.0, 1.0))
])

train_ds = LoadData(train, transform=tfms)
val_ds = LoadData(val, transform=tfms)

In [ ]:
for i, j in train_ds:
    print(i.shape, j)
    break
    
for i, j in val_ds:
    print(i.shape, j)
    break

In [8]:
bs = 32
trainloader = DataLoader(train_ds, batch_size=bs, shuffle=True)
valloader = DataLoader(val_ds, batch_size=bs, shuffle=False)

In [ ]:
for i, j in trainloader:
    print(i.shape, j.shape)
    break

for i, j in valloader:
    print(i.shape, j.shape)
    break

In [ ]:
def get_model(name, device=device):
    if name == 'efficientnet':
      model = EfficientNet.from_pretrained('efficientnet-b2')  
      for param in model.parameters():
        param.requires_grad = False
    
      model._fc = nn.Linear(1408, 1)
    
    return model.to(device)

In [ ]:
model = get_model('efficientnet')

In [ ]:
for param in model.parameters():
    if param.requires_grad == True:
        print(param.shape)

In [14]:
wt_0 = round(3878.0/5144.0, 3)
wt_1 = round(1266.0/5144.0, 3)

def weighted_bceloss(pred, label, weights=None): 
    return torch.where(
                       label==0, 
                       weights[0]*F.binary_cross_entropy_with_logits(pred, label), 
                       weights[1]*F.binary_cross_entropy_with_logits(pred, label)
    )

#criterion = nn.BCEWithLogitsLoss(weight=torch.tensor([wt_0, wt_1])) #weighted loss
opt = optim.Adam(model.parameters(), 1e-3)
scheduler = optim.lr_scheduler.CyclicLR(opt, base_lr=1e-3, max_lr=0.01, cycle_momentum=False)

NameError: name 'model' is not defined

In [ ]:
def validate(dataloader):
  correct = 0.0
  criterion = nn.BCEWithLogitsLoss()
  with torch.no_grad():
      for data in tqdm(dataloader, total=len(dataloader), leave=False):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        pred = (torch.sigmoid(outputs) >= 0.5).float()

        correct += (pred == labels).sum().item()
  total = len(dataloader)*bs

  return criterion(outputs, labels), (correct/total * 100)

In [ ]:
epochs = 5

for epoch in range(epochs):
    model.train()
    for data, labels in tqdm(trainloader, total=len(trainloader), leave=False):
        opt.zero_grad()
        
        out = model(data.to(device))
        loss = weighted_bceloss(out, labels.to(device), [wt_0, wt_1]).mean()
        loss.backward()
        
        opt.step()
        scheduler.step()
        
    validation = validate(valloader)
    
    print(f"Epochs: {epoch+1}/{epochs}\ttrain_loss: {loss.item()}\tval_loss: {validation[0].item()}\tacc: {validation[1]}")      

In [ ]:
def get_model_name(name, device=device):
    if name == 'efficientnet':
      model = EfficientNet.from_name('efficientnet-b2')  
      for param in model.parameters():
        param.requires_grad = False
    
      model._fc = nn.Linear(1408, 1)
    
    return model

In [ ]:
saved_model = '../input/model1/model-19.pt'

model = get_model_name('efficientnet')
model.load_state_dict(torch.load(saved_model, map_location='cpu'))
model.eval()

In [ ]:
p = '../input/aaaaaa/covid19_xray.webp'
im = tfms(Image.open(p).convert('RGB'))

with torch.no_grad():
  p = model(im.unsqueeze(0))
  print(torch.sigmoid(p))

In [ ]:
for params in model.parameters():
    params.requires_grad = True

opt1 = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
torch.save(model.state_dict(), 'model-19.pt')

# Pytorh lightning

In [9]:
from pytorch_lightning import LightningModule, Trainer
import torch.nn.functional as F

In [15]:
class Classifier(LightningModule):
    def __init__(self, model, lr, w_0, w_1):
        super().__init__()
        self.model = model
        self.save_hyperparameters()
        
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        out = self.model(x)
        loss = weighted_bceloss(out, y, [self.hparams.w_0, self.hparams.w_1]).mean()
        self.log('train_loss', loss, on_epoch=True)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        out = self.model(x)
        loss = weighted_bceloss(out, y, [self.hparams.w_0, self.hparams.w_1]).mean()
        self.log('val_loss', loss, on_epoch=True)
        
        return loss
    
    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.hparams.lr)  

In [16]:
wt_0 = round(3878.0/5144.0, 3)
wt_1 = round(1266.0/5144.0, 3)

def weighted_bceloss(pred, label, weights=None): 
    return torch.where(
                       label==0, 
                       weights[0]*F.binary_cross_entropy_with_logits(pred, label), 
                       weights[1]*F.binary_cross_entropy_with_logits(pred, label)
    )

In [17]:
def get_model_pl(name):
    if name == 'efficientnet':
      model = EfficientNet.from_pretrained('efficientnet-b2')  
      for param in model.parameters():
        param.requires_grad = False
    
      model._fc = nn.Linear(1408, 1)
    
    return model

In [18]:
classifier = Classifier(get_model_pl('efficientnet'), lr=1e-3, w_0=wt_0, w_1=wt_1)
trainer = Trainer(max_epochs=3,
                  gpus=1,
                  progress_bar_refresh_rate=60)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Loaded pretrained weights for efficientnet-b2


In [19]:
trainer.fit(classifier, trainloader, valloader)


  | Name  | Type         | Params
---------------------------------------
0 | model | EfficientNet | 7 M   


RuntimeError: grad can be implicitly created only for scalar outputs